

```
# Выбран кодовый формат
```

# Домашнее задание 2. Поисковая система для документов

**Модуль 2. Классический поиск и рекуррентные архитектуры**

**ФИО студента:** Майер Юрий Алексеевич

**Дата выполнения:** 28 сентября 25


## Описание задания

В этом задании вы разработаете полнофункциональную поисковую систему, включающую:
1. **Предобработку корпуса.**
2. **BM25.**
3. **Векторный поиск** — на основе эмбеддингов.
4. **Гибридный поиск** — комбинация BM25 и векторного поиска.
5. **Выбор метрики и оценку качества** — для конкретной задачи.

---

## Установка и импорт библиотек

In [1]:
import re
import math
import time
from typing import List, Dict, Tuple, Optional, Set
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3

# Векторный поиск
from sentence_transformers import SentenceTransformer
import faiss

# BM25
from rank_bm25 import BM25Okapi

# Инициализация морфологического анализатора
morph = pymorphy3.MorphAnalyzer()


## Часть 1. Подготовка данных

1. Загрузите и изучите предложенный датасет.  
2. Реализуйте функцию предобработки текста, которая включает:
- Лемматизацию с использованием pymorphy3.
- Удаление стоп-слов и пунктуации.  
3. Обработайте весь корпус документов и сохраните результат для последующих шагов.  


In [2]:
from datasets import load_dataset

# Загружаем корпус документов
ds = load_dataset("MLNavigator/russian-retrieval",
                  cache_dir='E:/hugging_face')
df = pd.DataFrame(ds['train'])
questions_df = df[['text','q']]


# Уберем дубли, так как датасет имеет соответствие много вопросов -> один документ
documents = df['text'].drop_duplicates().to_list()

In [3]:
nltk.download('punkt', download_dir=r"e:\nltk_data")

[nltk_data] Downloading package punkt to e:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('stopwords', download_dir=r"e:\nltk_data")

[nltk_data] Downloading package stopwords to e:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
nltk.data.path.insert(0, r"e:\nltk_data")

In [6]:
import string


russian_stopwords = set(stopwords.words("russian"))

In [7]:
def preprocess_text(text: str) -> str:
    """
    Базовая предобработка текста:
    1 Приведёём к нижнему регистру
    2. Токенизируем
    3. Фильтруем по пунктуации и стоп-словам
    4 Лемматизация
    """
    # В нижний регистр
    text = text.lower()
    
    # Токенизация
    tokens = word_tokenize(text, language="russian")
    
    cleaned_tokens = []
    for token in tokens:
        # Пропускаем пунктуацию и стоп-слова
        if token in russian_stopwords or token in string.punctuation:
            continue
        
        # Лемматизация
        lemma = morph.parse(token)[0].normal_form
        cleaned_tokens.append(lemma)
    
    return " ".join(cleaned_tokens)

In [8]:
# обернём в tqdm
processed_documents = []

for doc in tqdm(documents, desc="Preprocessing corpus"):
    processed_documents.append(preprocess_text(doc))


docs_df = pd.DataFrame({
    "original": documents,
    "processed": processed_documents
})

docs_df.head()


Preprocessing corpus:   0%|          | 0/9076 [00:00<?, ?it/s]

,original,processed
0,В протерозойских отложениях органические остат...,протерозойский отложение органический остаток ...
1,Кишечник млекопитающего подразделяется на тонк...,кишечник млекопитающее подразделяться тонкий т...
2,Город Байконур и космодром Байконур вместе обр...,город байконур космодром байконур вместе образ...
3,Вскоре после прибытия Колумба из Вест-Индии во...,вскоре прибытие колумб вест-индия возникнуть н...
4,Около Порт-Артура ночью на 27 января 1904 года...,около порт-артур ночью 27 январь 1904 год нача...


In [9]:
import os

os.mkdir('data')

In [10]:
docs_df.to_csv("data/processed_documents.csv", index=False, encoding="utf-8")

## Часть 2. Реализация BM25

1. Постройте инвертированный индекс для корпуса. Индекс должен содержать частоту термина в документе (TF) и документную частоту (DF).
2. Реализуйте функцию поиска BM25 с нуля. Формула для ранжирования:
score(D, Q) = Σ IDF(qi) * (f(qi, D) * (k1 + 1)) / (f(qi, D) + k1 * (1 - b + b * |D| / avgdl))
3. Проведите оптимизацию гиперпараметра k1, чтобы улучшить качество поиска.

Постриом инвертированный индекс:

In [12]:
from collections import defaultdict
import math

# Строим индекс по предобработанным документам
inverted_index = defaultdict(dict)  # term -> {doc_id: freq}
doc_lengths = []  # длины документов (в токенах)

for doc_id, doc in enumerate(docs_df["processed"]):
    tokens = doc.split()
    doc_lengths.append(len(tokens))
    freqs = Counter(tokens)
    for term, count in freqs.items():
        inverted_index[term][doc_id] = count

N = len(docs_df)    # общее число документов
avgdl = sum(doc_lengths) / N    # средняя длина документа

Создадим класс под BM25:

In [13]:
class BM25:
    def __init__(self, inverted_index, doc_lengths, avgdl, k1=1.5, b=0.75):
        self.index = inverted_index
        self.doc_lengths = doc_lengths
        self.avgdl = avgdl
        self.N = len(doc_lengths)
        self.k1 = k1
        self.b = b

    def idf(self, term):

        df = len(self.index.get(term, {}))
        if df == 0:
            return 0
        return math.log(1 + (self.N - df + 0.5) / (df + 0.5))

    def score(self, query, doc_id):
        tokens = query.split()
        score = 0.0
        for term in tokens:
            if doc_id not in self.index.get(term, {}):
                continue
            f = self.index[term][doc_id]  # term frequency
            idf = self.idf(term)
            dl = self.doc_lengths[doc_id]
            denom = f + self.k1 * (1 - self.b + self.b * dl / self.avgdl)
            score += idf * (f * (self.k1 + 1)) / denom
        return score

    def search(self, query, top_k=5):

        scores = []
        for doc_id in range(self.N):
            s = self.score(query, doc_id)
            if s > 0:
                scores.append((doc_id, s))
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:top_k]


In [14]:
bm25 = BM25(inverted_index, doc_lengths, avgdl, k1=1.5, b=0.75)

In [16]:
# Реализация поиска используя BM25

# Тестовые запросы
test_queries = [
    "машинное обучение",
    "нейронные сети",
    "поисковые системы BM25",
    "Python программирование"
]

print("Тестирование BM25:")
for query in test_queries:
    results = bm25.search(preprocess_text(query), top_k=3)
    print(f"Запрос: '{query}'")
    for doc_id, score in results:
        snippet = docs_df.loc[doc_id, "original"][:80].replace("\n", " ")
        print(f"  [{score:.3f}] {snippet}...")
    print()

Тестирование BM25:
Запрос: 'машинное обучение'
  [10.658] Особенность программного обеспечения состоит в том, что оно производится в одной...
  [10.270] Промышленный переворот, произошедший с 60-х годов XVIII до первой четверти XIX в...
  [8.999] Цифровые обучающие игры отличаются от традиционных обучающих игр и не основанног...

Запрос: 'нейронные сети'
  [18.523] Области мозга, постоянно используемые, когда человек занят вопросами морали, был...
  [7.732] Двусторонние рынки (двусторонние сети) — сетевые рынки, которые имеют две группы...
  [7.476] Во множестве сетей пользователи являются гомогенными, то есть они выполняют один...

Запрос: 'поисковые системы BM25'
  [17.158] Для поиска информации с помощью поисковой системы пользователь формулирует поиск...
  [16.063] Полезность поисковой системы зависит от релевантности найденных ею страниц. Хоть...
  [15.075] Когда пользователь вводит запрос в поисковую систему (обычно при помощи ключевых...

Запрос: 'Python программирование'
  [11.

Вроде находим по смыслу!

## Часть 3. Векторный поиск

1. Используйте предобученную модель sentence-transformers для получения векторных представлений (эмбеддингов) всех документов.
2. Создайте индекс для быстрого поиска ближайших соседей с помощью faiss-cpu.
3. Реализуйте функцию векторного поиска, которая по запросу находит top-k наиболее близких документов.


In [ ]:
model = SentenceTransformer('BAAI/bge-m3', device='cuda')


# Тестируем
print("Тестирование векторного поиска:")
for query in test_queries:
    results = vector_search.search(query, top_k=3)
    print(f"Запрос: '{query}'")
    for doc_id, score in results:
        doc = documents[doc_id]
        print(f"  [{score:.3f}] {doc.title[:50]}...")

## Часть 4. Гибридный поиск

1. Разработайте функцию, которая комбинирует результаты ранжирования от BM25 и векторного поиска.
2. Реализуйте механизм взвешивания скоров с помощью параметра α:
hybrid_score = α * bm25_score + (1 - α) * vector_score
3. Проведите автоматическую оптимизацию параметра α на валидационном наборе данных.


In [ ]:
# Тестируем гибридный поиск
print("Тестирование гибридного поиска:")
for query in test_queries:
    results = hybrid_search.search(query, top_k=3)
    print(f"Запрос: '{query}'")
    for doc_id, score in results:
        doc = documents[doc_id]
        print(f"  [{score:.3f}] {doc.title[:50]}...")

## Часть 5. Оценка качества

1. Выберите и **обоснуйте метрику** для оценки качества вашей поисковой системы (например, MRR, MAP@k или NDCG@k). **Обязательно подумайте о том, какой топ-к нужно выбрать исходя из данных**.
2. **Создайте небольшой датасет для оценки**, состоящий из запросов и релевантных им документов.  
3. **Сравните качество** всех трех реализованных подходов (BM25, векторный, гибридный) на вашем датасете.  
